# IMDB Movie Review Dataset

This is a library to download and parse the [IMDB's Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) dataset and a demo of a transformer based model. The dataset has 25K training, and 25K test dataset, plus 50K unlabeled examples.

It's inspired on [Keras' Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/) demo.




## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"

## Data Preparation

### Downloading data files

To download, uncompress and untar to the local directory, simply do the following. Notice if it's already downloaded in the given `--data` directory, it returns immediately.

In [2]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
    "github.com/gomlx/gomlx/ml/data"
)

var flagDataDir = flag.String("data", "~/work/imdb", "Directory to cache downloaded and generated dataset files.")

func AssertNoError(err error) {
    if err != nil {
        log.Fatalf("Failed: %+v", err)
    }
}

func AssertDownloaded() {
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    if !data.FileExists(*flagDataDir) {
        AssertNoError(os.MkdirAll(*flagDataDir, 0777))
    }

    AssertNoError(imdb.Download(*flagDataDir))
}

%%
AssertDownloaded()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.


### Sampling some examples

It creates a small dataset and print out some random examples.

It also defines the `DType`, used for all internal representations of the model, and the flag `--max_len` that defines the maximum number of tokens used per observation. This will beused in the modeling later.

In [3]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
)

// DType used for the models.
const DType = shapes.Float32

var (
    flagMaxLen              = flag.Int("max_len", 200, "Maximum number of tokens to take from observation.")
)

func Sample() {
    ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
    _, inputs, labels, err := ds.Yield()
    AssertNoError(err)
    labelsData := shapes.CastAsDType(labels[0].Local().Data(), shapes.Int64).([]int)
    for ii := 0; ii < 3; ii++ {
        fmt.Printf("\nlabel=%v, input=%q\n", labelsData[ii], imdb.InputToString(inputs[0], ii))
    }
    fmt.Println()
}

%% --max_len=200
AssertDownloaded()
Sample()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.

label=0, input="<START> tim curry was the reason i wanted to see this film and while of course his appearance is always entertaining he s basically wasted in it the rest of the cast doesn t fare too well either in this remake of an early zombie movie that has extremely graphic effects that are totally unnecessary to quote a popular axiom sometimes less is more"

label=1, input="good 1970s movie about mob operations in new jersey when a maverick gangster doesn t play by the rules of the neighborhood sooner or later it s time for elimination joe pesci was true to his character smooth and funny he only gets better with age his face and present day fame should not have been used on the dvd cover to sell this b grade movie as he was only the third billed star dated 1970 s printed wide lapel shirts and lesser quality background music make 

## Training

We will create 3 different types of models for this demo: **Bag of Words** (or simply **bow**), **CNNs** and **Transformers**.

### Model Support

We first define here a few componens shared among all models:

* `Normalize` according to flag, works for sequence nodes (shaped `[batch_size, sequence_len, embedding_dim]`) and feature nodes (shaped `[batch_size, embedding_dim]`).
* `EmbedTokensGraph` that embeds the tokens, before being consumed by the models.
* `ReadoutGraph` that takes the embeddings after they are pooled on the sequence axis (so shaped `[batch_size, embedding_dim]` it adds a FNN (feed-forward neural network) with a few layers and convert to the final logits.

We also define the corresponding hyperparameters.

In [4]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    flagNormalization    = flag.String("norm", "layer", "Type of normalization to use. Valid values are \"none\", \"batch\", \"layer\".")
    flagDropoutRate      = flag.Float64("dropout", 0.15, "Dropout rate")
    flagMaxVocab            = flag.Int("max_vocab", 20000, "Top most frequent words to consider, the rest is considered unknown.")
    flagTokenEmbeddingSize  = flag.Int("token_embed", 32, "Size of token embedding table. There are ~140K unique tokens")
    flagNumHiddenLayers     = flag.Int("hidden_layers", 2, "Number of output hidden layers, stacked with residual connection.")
    flagNumNodes            = flag.Int("num_nodes", 32, "Number of nodes in output hidden layers.")
    flagWordDropoutRate  = flag.Float64("word_dropout", 0, "Dropout rate for whole words of the input")
)

// Normalize x according to --norm flag. Works for sequence nodes (rank-3) or plain feature nodes (rank-2).
func Normalize(ctx *context.Context, x *Node) *Node {
    switch *flagNormalization {
    case "layer":
        if x.Rank() == 3 {
            // Normalize sequence.
            return layers.LayerNormalization(ctx, x, -2, -1).
                LearnedOffset(true).LearnedScale(true).ScaleNormalization(true).Done()
        } else {
            // Normalize features only.
            return layers.LayerNormalization(ctx, x, -1).Done()
        }
    case "batch":
        return layers.BatchNormalization(ctx, x, -1).Done()
    case "none":
        return x
    }
    g := x.Graph()
    g.SetErrorf("invalid normalization selected %q -- valid values are batch, layer, none", *flagNormalization)
    return g.InvalidNode()
}

// EmbedTokensGraph creates embeddings for tokens and returns them along with the mask of used tokens --
// set to false where padding was used.
func EmbedTokensGraph(ctx *context.Context, tokens *Node) (embed, mask *Node) {
    g := tokens.Graph()
    mask = NotEqual(tokens, ZerosLike(tokens)) // Mask of tokens actually used.

    // The token ids are indexed by frequency. Truncate to the vocabulary size considered, replacing
    // ids higher than that by 0.
    maxVocab := len(imdb.LoadedVocab.ListEntries)
    if maxVocab > *flagMaxVocab {
        maxVocab = *flagMaxVocab
    }

    // Limits tokens to the maxVocab.
    tokens = Where(GreaterOrEqual(tokens, Const(g, maxVocab)),
        MulScalar(OnesLike(tokens), float64(maxVocab-1)),
        tokens)

    // Embed tokens: shape=[batchSize, maxLen, embedDim]
    embed = layers.Embedding(ctx.In("tokens"), tokens, DType, maxVocab, *flagTokenEmbeddingSize)
    embed = Where(mask, embed, ZerosLike(embed))
    
    if *flagWordDropoutRate > 0 {
        dims := embed.Shape().Dimensions[:len(embed.Shape().Dimensions)-1]
        dropoutMask := Ones(g, shapes.Make(DType, dims...))
        dropoutMask = layers.Dropout(ctx, dropoutMask, ConstAsDType(g, DType, *flagWordDropoutRate))
        dropoutMask = ExpandDims(dropoutMask, -1)
        embed = Mul(embed, dropoutMask)
    }    
    return
}

// Activation function used for models.
func Activation(x *Node) *Node {
    return layers.Relu(x)
}

// ReadoutGraph takes the embeddings after they have been pooled on the sequence axis, so shaped `[batch_size, embed_dim]`
// adds a FNN on top and readout the final logit.
func ReadoutGraph(ctx *context.Context, embed *Node) *Node {
    g := embed.Graph()
    if !g.Ok() {
        return g.InvalidNode()
    }
    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Output layers.
    for ii := 0; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("output_dense_%d", ii))
        residual := embed
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        //embed = Tanh(embed)
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, *flagNumNodes)
        embed = Normalize(ctx, embed)
        if ii > 0 {
            // Add residual connection.
            embed = Add(embed, residual)
        }
    }

    // Final embed layer with dimension 1.
    {
        ctx := ctx.In("readout")
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, 1)
    }
    return embed
}

### Bag Of Words Model

This is the simples model we are going to train. It's basically a no-op that connects the embedding table we defined above with the `ReadoutGraph` FNN afterwards.

We define here placeholders for our future *CNN* and *Transformer* models.

Finally we test that the shape is correct. To actually train and evaluate we still need to define the training loop, what we do in the following section.

In [5]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
)

var (
    // ML Manager creation flags.
    flagNumThreads  = flag.Int("num_threads", -1, "Number of threads. Leave as -1 to use as many as there are cores.")
    flagNumReplicas = flag.Int("num_replicas", 1, "Number of replicas.")
    flagPlatform    = flag.String("platform", "", "Platform to use, if empty uses the default one.")

    // Model type.
    flagModel            = flag.String("model", "transformer", "Model type: bow or transformer.")
)

// ModelGraph builds the model for our demo. It returns the logits, not the predictions, which works with most losses.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
	_ = spec // Not used.
	tokens := inputs[0]
	g := tokens.Graph()
	if !g.Ok() {
		return nil
	}
	embed, mask := EmbedTokensGraph(ctx, tokens)

	// Normalization function.
	if *flagModel == "bow" {
		// Bag-Of-Words model doesn't do anything, it's just the embedding table for each token.
	} else if *flagModel == "cnn" {
		embed = Conv1DGraph(ctx, embed, mask)
	} else if *flagModel == "transformer" {
		embed = TransformerGraph(ctx, tokens, embed, mask)
	} else {
		g.SetErrorf("unknown model type %q, only types \"bow\", \"cnn\" and \"transformer\" are implemented", *flagModel)
		return nil
	}

	// Sum-up per-token embeddings and do a FNN on the output. From now on, the dimensions are `[batch_dim, embed_dim]`
	// Notice we are not using mask.
	embed = ReduceMax(embed, 1)
	logits := ReadoutGraph(ctx, embed)
	return []*Node{logits}
}

func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    log.Panic("Not implemented.")
    return nil
}

func TransformerGraph(ctx *context.Context, tokens, embed, mask *Node) *Node {
    log.Panic("Not implemented.")
    return nil
}

%% --model=bow --platform=Host

// Let's test that the logits are coming out with the right shape: we want [batch_size, 10], since there are 10 classes.
AssertDownloaded()
manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
_, inputs, _, err := ds.Yield()
AssertNoError(err)

g := manager.NewGraph("test")
ctx := context.NewContext(manager)
logits := ModelGraph(ctx, nil, []*Node{Const(g, inputs[0])})
AssertNoError(g.Error())
AssertNoError(ctx.Error())
fmt.Printf("Logits shape for batch_size=%d: %s\n", 3, logits[0].Shape())

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Logits shape for batch_size=3: (Float32)[3 1]


### Training Loop

Training loop, with the usual bells and whistles. We also do a quick test with 100 steps to check things are working.

In [6]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
)

var (
    flagOptimizer        = flag.String("optimizer", "adam", "Optimizer, options: adam or sgd.")
    flagNumSteps         = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
    flagBatchSize        = flag.Int("batch", 32, "Batch size for training")
    flagLearningRate     = flag.Float64("learning_rate", 0.0001, "Initial learning rate.")
    flagL2Regularization = flag.Float64("l2_reg", 0, "L2 regularization on kernels. It doesn't interact well with --batch_norm or with --optimizer=adam.")
    
    flagCheckpoint       = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep   = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")

    // Training UI:
    flagUseProgressBar = flag.Bool("bar", true, "If to display a progress bar during training")
    flagNumPlotPoints  = flag.Int("plot_points", 50, "Number points to plot.")
)

func trainModel() {
    // Make sure data is downloaded.
    AssertDownloaded()

    // Manager handles creation of ML computation graphs, accelerator resources, etc.
    manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
    fmt.Printf("Platform: %s\n", manager.Platform())

    // Datasets.
    var trainDS, trainEvalDS, testEvalDS train.Dataset
    trainDS = imdb.NewDataset("train", imdb.Train, *flagMaxLen, *flagBatchSize, DType, true, nil)
    trainEvalDS = imdb.NewDataset("train-eval", imdb.Train, *flagMaxLen, *flagBatchSize, DType, false, nil)
    testEvalDS = imdb.NewDataset("test-eval", imdb.Test, *flagMaxLen, *flagBatchSize, DType, false, nil)

    // Parallelize generation of batches, to prevent the dataset from being a bottleneck to the acceleartor (GPU/TPU).
    trainDS = data.NewParallelDataset(trainDS, 0, 0)
    trainEvalDS = data.NewParallelDataset(trainEvalDS, 0, 0)
    testEvalDS = data.NewParallelDataset(testEvalDS, 0, 0)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Context holds the variables and hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)
    ctx.SetParam(layers.L2RegularizationKey, *flagL2Regularization)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        checkpointPath := data.ReplaceTildeInDir(*flagCheckpoint)
        if !path.IsAbs(checkpointPath) {
            checkpointPath = path.Join(*flagDataDir, checkpointPath)
        }
        var err error
        checkpoint, err = checkpoints.Build(ctx).Dir(checkpointPath).Keep(*flagCheckpointKeep).Done()
        AssertNoError(err)
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    loss := losses.BinaryCrossentropyLogits
    trainer := train.NewTrainer(
        manager, ctx, ModelGraph, loss,
        optimizers.MustOptimizerByName(*flagOptimizer),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    if *flagUseProgressBar {
        commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.
    }

    if checkpoint != nil {
        train.NTimesDuringLoop(loop, 2*(*flagCheckpointKeep), "checkpointing", 100, func(_ *train.Loop, _ []tensor.Tensor) error {
            return checkpoint.Save()
        })
    }

    // Attach a margaid plots.
    if *flagNumPlotPoints > 0 {
        margaid.New(1024, 400, trainEvalDS, testEvalDS).DynamicUpdates().Attach(loop, *flagNumPlotPoints)
    }

    // Loop for given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    AssertNoError(err)

    // Report final evaluation.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    AssertNoError(err)
}

%% --platform=Host --model=bow --steps=100 --plot_points=0
trainModel()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: Host
Training (100 steps):  100% [========================================] (146 steps/s) [loss=0.710] [~loss=0.687] [~acc=55.72%]        

Results on train-eval:
	Mean Loss (#loss): 0.700
	Mean Accuracy (#acc): 50.01%
Results on test-eval:
	Mean Loss (#loss): 0.700
	Mean Accuracy (#acc): 50.00%


### Training Bag-Of-Words ("bow") Model

A proper run of our *bow* model.

In [7]:
%% --model=bow --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (707 steps/s) [loss=0.218] [~loss=0.232] [~acc=91.03%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.461121e+01,5.200000e+00 L5.158486e+01,1.178466e+02 L6.855851e+01,1.322440e+02 L8.553216e+01,1.652826e+02 L1.025058e+02,1.684154e+02 L1.194795e+02,1.682821e+02 L1.364531e+02,1.767472e+02 L1.534268e+02,1.812131e+02 L1.704004e+02,1.812131e+02 L1.873741e+02,1.859678e+02 L2.043477e+02,1.865677e+02 L2.213214e+02,1.871009e+02 L2.382950e+02,1.873898e+02 L2.552686e+02,1.866788e+02 L2.722423e+02,1.871898e+02 L2.892159e+02,1.899449e+02 L3.061896e+02,1.930999e+02 L3.231632e+02,1.914779e+02 L3.401369e+02,1.934109e+02 L3.571105e+02,1.940330e+02 L3.740842e+02,1.943219e+02 L3.910578e+02,1.912558e+02 L4.080315e+02,1.948774e+02 L4.250051e+02,1.937887e+02 L4.419788e+02,1.958327e+02 L4.589524e+02,1.945441e+02 L4.759261e+02,1.980768e+02 L4.928997e+02,1.955217e+02 L5.098734e+02,1.988988e+02 L5.268470e+02,1.978990e+02 L5.438207e+02,1.982545e+02 L5.607943e+02,1.979434e+02 L5.777680e+02,1.988100e+02 L5.947416e+02,1.981879e+02 L6.117153e+02,1.930332e+02 L6.286889e+02,1.954328e+02 L6.456626e+02,1.960771e+02 L6.626362e+02,1.983878e+02 L6.796099e+02,1.965215e+02 L6.965835e+02,1.988544e+02 L7.135572e+02,1.964326e+02 L7.305308e+02,1.975657e+02 L7.475045e+02,1.913446e+02 L7.644781e+02,1.973880e+02 L7.814518e+02,1.973213e+02 L7.984254e+02,1.941886e+02 L8.153991e+02,1.956106e+02 L8.323727e+02,1.958105e+02 L8.493464e+02,1.995432e+02 L8.663200e+02,1.955217e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.222208e+00 L3.461121e+01,5.222208e+00 L5.158486e+01,1.226458e+02 L6.855851e+01,1.352435e+02 L8.553216e+01,1.735478e+02 L1.025058e+02,1.767472e+02 L1.194795e+02,1.774138e+02 L1.364531e+02,1.897227e+02 L1.534268e+02,1.975213e+02 L1.704004e+02,1.960771e+02 L1.873741e+02,2.064531e+02 L2.043477e+02,2.052533e+02 L2.213214e+02,2.060976e+02 L2.382950e+02,2.070307e+02 L2.552686e+02,2.069418e+02 L2.722423e+02,2.082305e+02 L2.892159e+02,2.123631e+02 L3.061896e+02,2.176733e+02 L3.231632e+02,2.165179e+02 L3.401369e+02,2.201617e+02 L3.571105e+02,2.221392e+02 L3.740842e+02,2.235167e+02 L3.910578e+02,2.210949e+02 L4.080315e+02,2.244721e+02 L4.250051e+02,2.259607e+02 L4.419788e+02,2.286269e+02 L4.589524e+02,2.280936e+02 L4.759261e+02,2.339148e+02 L4.928997e+02,2.313153e+02 L5.098734e+02,2.363144e+02 L5.268470e+02,2.362922e+02 L5.438207e+02,2.382029e+02 L5.607943e+02,2.389139e+02 L5.777680e+02,2.413135e+02 L5.947416e+02,2.406914e+02 L6.117153e+02,2.337815e+02 L6.286889e+02,2.370698e+02 L6.456626e+02,2.393139e+02 L6.626362e+02,2.435798e+02 L6.796099e+02,2.418912e+02 L6.965835e+02,2.465348e+02 L7.135572e+02,2.445129e+02 L7.305308e+02,2.472680e+02 L7.475045e+02,2.400693e+02 L7.644781e+02,2.492899e+02 L7.814518e+02,2.486011e+02 L7.984254e+02,2.447573e+02 L8.153991e+02,2.486455e+02 L8.323727e+02,2.485567e+02 L8.493464e+02,2.548000e+02 L8.663200e+02,2.514228e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.257467e+01 L3.461121e+01,9.914862e+00 L5.158486e+01,2.184895e+01 L6.855851e+01,5.572410e+01 L8.553216e+01,1.131721e+02 L1.025058e+02,1.302958e+02 L1.194795e+02,1.456957e+02 L1.364531e+02,1.451414e+02 L1.534268e+02,1.546292e+02 L1.704004e+02,1.581200e+02 L1.873741e+02,1.673895e+02 L2.043477e+02,1.667235e+02 L2.213214e+02,1.729198e+02 L2.382950e+02,1.726124e+02 L2.552686e+02,1.789105e+02 L2.722423e+02,1.806479e+02 L2.892159e+02,1.808205e+02 L3.061896e+02,1.877836e+02 L3.231632e+02,1.888142e+02 L3.401369e+02,1.950412e+02 L3.571105e+02,1.992844e+02 L3.740842e+02,1.977700e+02 L3.910578e+02,2.018441e+02 L4.080315e+02,1.990708e+02 L4.250051e+02,2.014368e+02 L4.419788e+02,2.028730e+02 L4.589524e+02,2.020081e+02 L4.759261e+02,2.030280e+02 L4.928997e+02,2.058773e+02 L5.098734e+02,2.088824e+02 L5.268470e+02,2.088254e+02 L5.438207e+02,2.079642e+02 L5.607943e+02,2.134199e+02 L5.777680e+02,2.134245e+02 L5.947416e+02,2.149332e+02 L6.117153e+02,2.122781e+02 L6.286889e+02,2.168310e+02 L6.456626e+02,2.178427e+02 L6.626362e+02,2.180574e+02 L6.796099e+02,2.205919e+02 L6.965835e+02,2.237183e+02


Results on train-eval:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.496787e+02 L3.461121e+01,2.488093e+02 L5.158486e+01,2.458779e+02 L6.855851e+01,2.246729e+02 L8.553216e+01,1.637043e+02 L1.025058e+02,1.520574e+02 L1.194795e+02,1.497707e+02 L1.364531e+02,1.399343e+02 L1.534268e+02,1.321936e+02 L1.704004e+02,1.322097e+02 L1.873741e+02,1.247612e+02 L2.043477e+02,1.233603e+02 L2.213214e+02,1.221928e+02 L2.382950e+02,1.211980e+02 L2.552686e+02,1.223525e+02 L2.722423e+02,1.210924e+02 L2.892159e+02,1.174039e+02 L3.061896e+02,1.126759e+02 L3.231632e+02,1.150953e+02 L3.401369e+02,1.117842e+02 L3.571105e+02,1.113836e+02 L3.740842e+02,1.112307e+02 L3.910578e+02,1.151750e+02 L4.080315e+02,1.105155e+02 L4.250051e+02,1.114456e+02 L4.419788e+02,1.104627e+02 L4.589524e+02,1.099787e+02 L4.759261e+02,1.051457e+02 L4.928997e+02,1.101854e+02 L5.098734e+02,1.060719e+02 L5.268470e+02,1.065351e+02 L5.438207e+02,1.057456e+02 L5.607943e+02,1.073233e+02 L5.777680e+02,1.051671e+02 L5.947416e+02,1.072211e+02 L6.117153e+02,1.139843e+02 L6.286889e+02,1.132968e+02 L6.456626e+02,1.113512e+02 L6.626362e+02,1.062254e+02 L6.796099e+02,1.117278e+02 L6.965835e+02,1.079483e+02 L7.135572e+02,1.131209e+02 L7.305308e+02,1.123220e+02 L7.475045e+02,1.187866e+02 L7.644781e+02,1.100888e+02 L7.814518e+02,1.096240e+02 L7.984254e+02,1.165923e+02 L8.153991e+02,1.156982e+02 L8.323727e+02,1.159297e+02 L8.493464e+02,1.081170e+02 L8.663200e+02,1.181250e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.496641e+02 L3.461121e+01,2.487300e+02 L5.158486e+01,2.456606e+02 L6.855851e+01,2.233620e+02 L8.553216e+01,1.580733e+02 L1.025058e+02,1.438085e+02 L1.194795e+02,1.391099e+02 L1.364531e+02,1.269016e+02 L1.534268e+02,1.160361e+02 L1.704004e+02,1.151225e+02 L1.873741e+02,1.029607e+02 L2.043477e+02,1.021585e+02 L2.213214e+02,9.888995e+01 L2.382950e+02,9.678128e+01 L2.552686e+02,9.457965e+01 L2.722423e+02,9.268474e+01 L2.892159e+02,8.739012e+01 L3.061896e+02,8.026744e+01 L3.231632e+02,8.182804e+01 L3.401369e+02,7.618851e+01 L3.571105e+02,7.178461e+01 L3.740842e+02,7.012819e+01 L3.910578e+02,7.255743e+01 L4.080315e+02,6.777719e+01 L4.250051e+02,6.578806e+01 L4.419788e+02,6.238358e+01 L4.589524e+02,6.190063e+01 L4.759261e+02,5.509803e+01 L4.928997e+02,5.791028e+01 L5.098734e+02,5.032907e+01 L5.268470e+02,5.073747e+01 L5.438207e+02,4.815916e+01 L5.607943e+02,4.551200e+01 L5.777680e+02,4.328997e+01 L5.947416e+02,4.283033e+01 L6.117153e+02,5.202358e+01 L6.286889e+02,4.596901e+01 L6.456626e+02,4.215263e+01 L6.626362e+02,3.813808e+01 L6.796099e+02,3.854887e+01 L6.965835e+02,3.345072e+01 L7.135572e+02,3.527740e+01 L7.305308e+02,3.151061e+01 L7.475045e+02,3.906295e+01 L7.644781e+02,2.873540e+01 L7.814518e+02,2.864543e+01 L7.984254e+02,3.250033e+01 L8.153991e+02,2.657871e+01 L8.323727e+02,2.633184e+01 L8.493464e+02,2.010891e+01 L8.663200e+02,2.230533e+01 "/> <path d="M1.768000e+01,2.548000e+02 L3.461121e+01,2.523718e+02 L5.158486e+01,2.427551e+02 L6.855851e+01,2.127093e+02 L8.553216e+01,1.730220e+02 L1.025058e+02,1.937740e+02 L1.194795e+02,2.134202e+02 L1.364531e+02,1.416539e+02 L1.534268e+02,1.190785e+02 L1.704004e+02,1.677888e+02 L1.873741e+02,1.891450e+02 L2.043477e+02,1.489364e+02 L2.213214e+02,1.079832e+02 L2.382950e+02,1.767679e+02 L2.552686e+02,1.482311e+02 L2.722423e+02,1.691437e+02 L2.892159e+02,8.788236e+01 L3.061896e+02,7.563612e+01 L3.231632e+02,1.281884e+02 L3.401369e+02,1.189454e+02 L3.571105e+02,2.067755e+02 L3.740842e+02,1.186703e+02 L3.910578e+02,8.252262e+01 L4.080315e+02,6.781376e+01 L4.250051e+02,8.183106e+01 L4.419788e+02,1.270668e+02 L4.589524e+02,1.494359e+02 L4.759261e+02,9.854346e+01 L4.928997e+02,8.515869e+01 L5.098734e+02,1.750925e+02 L5.268470e+02,8.056359e+01 L5.438207e+02,1.522597e+02 L5.607943e+02,1.327977e+01 L5.777680e+02,1.099301e+02 L5.947416e+02,1.010897e+02 L6.117153e+02,1.375023e+02 L6.286889e+02,1.473044e+02 L6.456626e+02,1.248314e+01 L6.626362e+02,4.227524e+01 L6.796099e+02,1.059114e+02 L6.965835e+02,3.762680e+01 L7.135572e+02,6.771518e+01 L7.3053

	Mean Loss (#loss): 0.154
	Mean Accuracy (#acc): 94.35%
Results on test-eval:
	Mean Loss (#loss): 0.382
	Mean Accuracy (#acc): 84.26%


### CNN Model

Below we redefine our CNN model function, with 2 convolution layers, and train on it.

Notice how well it can overfit to the training data ... but it doesn't help the test results. To improve this one needs some careful regularization.

In [8]:
// Conv1DGraph stacks two 1D convolution layers on top of the sequence embeddings (shaped `[batch_size, sequence_len, embed_dim]`).
func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    // 1D Convolution:
    {
        ctx := ctx.In("conv1")
        embed = Activation(embed)
        embed = layers.Dropout(ctx, embed, ConstAsDType(g, DType, *flagDropoutRate))
        embed = layers.Convolution(ctx, embed).ChannelsAfter().KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        AssertNoError(g.Error())
        embed = Normalize(ctx, embed)
    }
    {
        ctx := ctx.In("conv2")
        embed = Activation(embed)
        embed = layers.Convolution(ctx, embed).ChannelsAfter().KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        AssertNoError(g.Error())
        embed = Normalize(ctx, embed)
    }
    return embed
}

%% --model=cnn --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (459 steps/s) [loss=0.134] [~loss=0.091] [~acc=96.88%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.560403e+01 L3.461121e+01,5.562016e+01 L5.158486e+01,5.565244e+01 L6.855851e+01,6.528470e+01 L8.553216e+01,5.879865e+01 L1.025058e+02,7.109310e+01 L1.194795e+02,9.192268e+01 L1.364531e+02,1.337754e+02 L1.534268e+02,1.471186e+02 L1.704004e+02,1.578481e+02 L1.873741e+02,1.673513e+02 L2.043477e+02,1.638662e+02 L2.213214e+02,1.759025e+02 L2.382950e+02,1.789519e+02 L2.552686e+02,1.810494e+02 L2.722423e+02,1.829533e+02 L2.892159e+02,1.833728e+02 L3.061896e+02,1.852605e+02 L3.231632e+02,1.817755e+02 L3.401369e+02,1.875838e+02 L3.571105e+02,1.874064e+02 L3.740842e+02,1.852121e+02 L3.910578e+02,1.857445e+02 L4.080315e+02,1.879549e+02 L4.250051e+02,1.879711e+02 L4.419788e+02,1.858091e+02 L4.589524e+02,1.894071e+02 L4.759261e+02,1.888585e+02 L4.928997e+02,1.875032e+02 L5.098734e+02,1.882292e+02 L5.268470e+02,1.886326e+02 L5.438207e+02,1.889069e+02 L5.607943e+02,1.889876e+02 L5.777680e+02,1.883744e+02 L5.947416e+02,1.876968e+02 L6.117153e+02,1.883583e+02 L6.286889e+02,1.879872e+02 L6.456626e+02,1.848733e+02 L6.626362e+02,1.873903e+02 L6.796099e+02,1.855832e+02 L6.965835e+02,1.877291e+02 L7.135572e+02,1.874225e+02 L7.305308e+02,1.875193e+02 L7.475045e+02,1.867933e+02 L7.644781e+02,1.854380e+02 L7.814518e+02,1.873741e+02 L7.984254e+02,1.863415e+02 L8.153991e+02,1.697714e+02 L8.323727e+02,1.872289e+02 L8.493464e+02,1.880033e+02 L8.663200e+02,1.875355e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.562016e+01 L3.461121e+01,5.558790e+01 L5.158486e+01,5.565244e+01 L6.855851e+01,6.994757e+01 L8.553216e+01,6.123496e+01 L1.025058e+02,8.506554e+01 L1.194795e+02,1.151079e+02 L1.364531e+02,1.576383e+02 L1.534268e+02,1.768383e+02 L1.704004e+02,1.905203e+02 L1.873741e+02,2.007980e+02 L2.043477e+02,1.984101e+02 L2.213214e+02,2.138185e+02 L2.382950e+02,2.161257e+02 L2.552686e+02,2.210951e+02 L2.722423e+02,2.249028e+02 L2.892159e+02,2.248061e+02 L3.061896e+02,2.289203e+02 L3.231632e+02,2.281136e+02 L3.401369e+02,2.341156e+02 L3.571105e+02,2.352773e+02 L3.740842e+02,2.344544e+02 L3.910578e+02,2.355193e+02 L4.080315e+02,2.391657e+02 L4.250051e+02,2.400531e+02 L4.419788e+02,2.400047e+02 L4.589524e+02,2.435704e+02 L4.759261e+02,2.446030e+02 L4.928997e+02,2.439899e+02 L5.098734e+02,2.453613e+02 L5.268470e+02,2.463455e+02 L5.438207e+02,2.481203e+02 L5.607943e+02,2.487012e+02 L5.777680e+02,2.493304e+02 L5.947416e+02,2.489432e+02 L6.117153e+02,2.504760e+02 L6.286889e+02,2.507986e+02 L6.456626e+02,2.488141e+02 L6.626362e+02,2.509761e+02 L6.796099e+02,2.510407e+02 L6.965835e+02,2.523476e+02 L7.135572e+02,2.527187e+02 L7.305308e+02,2.531220e+02 L7.475045e+02,2.527509e+02 L7.644781e+02,2.527025e+02 L7.814518e+02,2.538481e+02 L7.984254e+02,2.538158e+02 L8.153991e+02,2.403919e+02 L8.323727e+02,2.538965e+02 L8.493464e+02,2.546387e+02 L8.663200e+02,2.548000e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.461121e+01,6.173617e+01 L5.158486e+01,5.432729e+01 L6.855851e+01,5.517047e+01 L8.553216e+01,5.707427e+01 L1.025058e+02,6.374654e+01 L1.194795e+02,9.284080e+01 L1.364531e+02,1.229025e+02 L1.534268e+02,1.495686e+02 L1.704004e+02,1.676641e+02 L1.873741e+02,1.845649e+02 L2.043477e+02,1.871872e+02 L2.213214e+02,1.921725e+02 L2.382950e+02,1.994514e+02 L2.552686e+02,2.019785e+02 L2.722423e+02,2.084576e+02 L2.892159e+02,2.104708e+02 L3.061896e+02,2.129985e+02 L3.231632e+02,2.141438e+02 L3.401369e+02,2.157001e+02 L3.571105e+02,2.166136e+02 L3.740842e+02,2.193370e+02 L3.910578e+02,2.237934e+02 L4.080315e+02,2.221473e+02 L4.250051e+02,2.250739e+02 L4.419788e+02,2.260791e+02 L4.589524e+02,2.294295e+02 L4.759261e+02,2.274457e+02 L4.928997e+02,2.304763e+02 L5.098734e+02,2.319998e+02 L5.268470e+02,2.318090e+02 L5.438207e+02,2.314306e+02 L5.607943e+02,2.348319e+02 L5.777680e+02,2.315715e+02 L5.947416e+02,2.342980e+02 L6.117153e+02,2.372734e+02 L6.286889e+02,2.382629e+02 L6.456626e+02,2.366937e+02 L6.626362e+02,2.401874e+02 L6.796099e+02,2.377645e+02 L6.965835e+02,2.382938e+02


Results on train-eval:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.179125e+02 L3.461121e+01,2.148834e+02 L5.158486e+01,2.135134e+02 L6.855851e+01,2.129344e+02 L8.553216e+01,2.128363e+02 L1.025058e+02,2.119353e+02 L1.194795e+02,2.046206e+02 L1.364531e+02,1.807237e+02 L1.534268e+02,1.704838e+02 L1.704004e+02,1.614137e+02 L1.873741e+02,1.535064e+02 L2.043477e+02,1.601846e+02 L2.213214e+02,1.407561e+02 L2.382950e+02,1.401457e+02 L2.552686e+02,1.366965e+02 L2.722423e+02,1.401846e+02 L2.892159e+02,1.355259e+02 L3.061896e+02,1.350859e+02 L3.231632e+02,1.438698e+02 L3.401369e+02,1.353174e+02 L3.571105e+02,1.343065e+02 L3.740842e+02,1.431523e+02 L3.910578e+02,1.468578e+02 L4.080315e+02,1.365824e+02 L4.250051e+02,1.449692e+02 L4.419788e+02,1.509114e+02 L4.589524e+02,1.458756e+02 L4.759261e+02,1.417342e+02 L4.928997e+02,1.516517e+02 L5.098734e+02,1.510966e+02 L5.268470e+02,1.578192e+02 L5.438207e+02,1.474788e+02 L5.607943e+02,1.500222e+02 L5.777680e+02,1.476339e+02 L5.947416e+02,1.524933e+02 L6.117153e+02,1.585817e+02 L6.286889e+02,1.647486e+02 L6.456626e+02,1.784689e+02 L6.626362e+02,1.683109e+02 L6.796099e+02,1.701275e+02 L6.965835e+02,1.681005e+02 L7.135572e+02,1.631469e+02 L7.305308e+02,1.661234e+02 L7.475045e+02,1.827275e+02 L7.644781e+02,1.879965e+02 L7.814518e+02,1.796628e+02 L7.984254e+02,1.749607e+02 L8.153991e+02,2.548000e+02 L8.323727e+02,1.829690e+02 L8.493464e+02,1.827399e+02 L8.663200e+02,1.837096e+02 "/> <path d="M1.768000e+01,2.180263e+02 L3.461121e+01,2.148285e+02 L5.158486e+01,2.133252e+02 L6.855851e+01,2.126689e+02 L8.553216e+01,2.124306e+02 L1.025058e+02,2.106136e+02 L1.194795e+02,1.969095e+02 L1.364531e+02,1.617155e+02 L1.534268e+02,1.386992e+02 L1.704004e+02,1.209620e+02 L1.873741e+02,1.055763e+02 L2.043477e+02,1.065153e+02 L2.213214e+02,8.728948e+01 L2.382950e+02,8.149682e+01 L2.552686e+02,7.494129e+01 L2.722423e+02,6.856006e+01 L2.892159e+02,6.600216e+01 L3.061896e+02,6.021444e+01 L3.231632e+02,6.136479e+01 L3.401369e+02,5.210852e+01 L3.571105e+02,5.008939e+01 L3.740842e+02,5.047928e+01 L3.910578e+02,4.823589e+01 L4.080315e+02,4.234205e+01 L4.250051e+02,4.013979e+01 L4.419788e+02,3.983593e+01 L4.589524e+02,3.347075e+01 L4.759261e+02,3.222497e+01 L4.928997e+02,3.230609e+01 L5.098734e+02,2.945392e+01 L5.268470e+02,2.753742e+01 L5.438207e+02,2.495037e+01 L5.607943e+02,2.357405e+01 L5.777680e+02,2.244846e+01 L5.947416e+02,2.241439e+01 L6.117153e+02,1.977090e+01 L6.286889e+02,1.858086e+01 L6.456626e+02,2.162653e+01 L6.626362e+02,1.731867e+01 L6.796099e+02,1.773130e+01 L6.965835e+02,1.445358e+01 L7.135572e+02,1.436009e+01 L7.305308e+02,1.291003e+01 L7.475045e+02,1.321864e+01 L7.644781e+02,1.321530e+01 L7.814518e+02,1.099968e+01 L7.984254e+02,1.176555e+01 L8.153991e+02,3.451607e+01 L8.323727e+02,1.054193e+01 L8.493464e+02,9.107655e+00 L8.663200e+02,8.746524e+00 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.353856e+02 L3.461121e+01,2.057216e+02 L5.158486e+01,2.176943e+02 L6.855851e+01,2.139907e+02 L8.553216e+01,2.150657e+02 L1.025058e+02,2.200149e+02 L1.194795e+02,2.072368e+02 L1.364531e+02,1.912339e+02 L1.534268e+02,1.594811e+02 L1.704004e+02,1.379080e+02 L1.873741e+02,1.514182e+02 L2.043477e+02,7.047324e+01 L2.213214e+02,1.658757e+02 L2.382950e+02,6.068332e+01 L2.552686e+02,1.060019e+02 L2.722423e+02,7.742411e+01 L2.892159e+02,1.047113e+02 L3.061896e+02,8.850834e+01 L3.231632e+02,3.126729e+01 L3.401369e+02,1.531863e+02 L3.571105e+02,1.043607e+02 L3.740842e+02,8.524780e+01 L3.910578e+02,6.685830e+01 L4.080315e+02,3.351854e+01 L4.250051e+02,8.113557e+01 L4.419788e+02,6.854980e+01 L4.589524e+02,2.200292e+01 L4.759261e+02,7.966917e+01 L4.928997e+02,3.482626e+01 L5.098734e+02,4.558780e+01 L5.268470e+02,6.506591e+01 L5.438207e+02,4.400525e+01 L5.607943e+02,5.434063e+01 L5.777680e+02,3.175552e+01 L5.947416e+02,1.494802e+01 L6.117153e+02,1.019438e+02 L6.286889e+02,2.162287e+01 L6.456626e+02,9.710454e+01 L6.626362e+02,3.934267e+01 L6.796099e+02,4.365786e+01 L6.965835e+02,5.436977e+01

	Mean Loss (#loss): 0.027
	Mean Accuracy (#acc): 99.40%
Results on test-eval:
	Mean Loss (#loss): 0.597
	Mean Accuracy (#acc): 82.71%


### Transformer Model

Finally a Transformer version of the model. 

Notice it's not significantly better than our previous simple Bag-Of-Words model. Likely because there is not enough data for the transformer to make any difference. The success of transformers in large-language-models is in large part due to the training with huge amounts of unsupervised (or self-supervised) data.


In [10]:
var (
	flagMaxAttLen           = flag.Int("max_att_len", 200, "Maximum attention length: input will be split in ranges of this size.")
	flagNumAttHeads         = flag.Int("att_heads", 2, "Number of attention heads, if --model=transformer.")
	flagNumAttLayers        = flag.Int("att_layers", 1, "Number of stacked attention layers, if --model=transformer.")
	flagAttKeyQueryEmbedDim = flag.Int("att_key_dim", 8, "Dimension of the Key/Query attention embedding.")    
)

// TransformerGraph is the part of the model that takes the word/token embeddings to a tranformed
// embedding through attention ready to be pooled and read out.
func TransformerGraph(ctx *context.Context, input, embed, mask *Node) *Node {
    var newEmbed *Node
    if *flagMaxAttLen >= *flagMaxLen {
        // One transformer window covers the whole length, which makes it trivial.
        newEmbed = TransformerLayers(ctx.In("transformer"), embed, mask)
        embed = Add(embed, newEmbed)
        return embed
    }

    // Split embedding in multiple split embeddings and apply transformer in each of them.
    attLen := *flagMaxAttLen
    sequenceFrom := 0
    for {
        // x.shape = [batchSize, sequence, embedding]
        sequenceTo := sequenceFrom + attLen
        if sequenceTo > *flagMaxLen {
            sequenceTo = *flagMaxLen
            sequenceFrom = sequenceTo - attLen
        }
        // part = x[:, sequenceFrom:sequenceTo, :]
        residual := Slice(embed, AxisRange(), AxisRange(sequenceFrom, sequenceTo), AxisRange())
        partMask := Slice(mask, AxisRange(), AxisRange(sequenceFrom, sequenceTo))
        // Reuse "transformer" scope.
        part := TransformerLayers(ctx.In("transformer").Checked(false), residual, partMask)
        part = Add(residual, part)
        if newEmbed == nil {
            newEmbed = part
        } else {
            newEmbed = Add(newEmbed, part)
        }

        if sequenceTo == *flagMaxLen {
            // Reached end of parts.
            break
        }
        sequenceFrom += attLen - 20 // Attention window overlap 10 tokens among themselves.
    }
    embed = newEmbed // Notice shape changed to `[batchSize, maxAttLen, embedDim]`
    return embed
}

// TransformerLayers builds the stacked transformer layers for the model.
func TransformerLayers(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    shape := embed.Shape()
    embedDim := shape.Dimensions[2]

    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Create positional embedding variable: it is 1 in every axis, but for the
    // sequence dimension -- there will be one embedding per position.
    // Shape: [1, maxLen, embedDim]
    posEmbedShape := shape.Copy()
    posEmbedShape.Dimensions[0] = 1
    posEmbedVar := ctx.VariableWithShape("positional", posEmbedShape)
    posEmbed := posEmbedVar.ValueGraph(g)
    embed = Add(embed, posEmbed) // Just add the embeddings, seems to work well.

    // Add the requested number of attention layers.
    for ii := 0; ii < *flagNumAttLayers; ii++ {
        // Each layer in its own scope.
        ctx := ctx.In(fmt.Sprintf("AttLayer_%d", ii))
        residual := embed
        embed = layers.MultiHeadAttention(ctx, embed, embed, embed, *flagNumAttHeads, *flagAttKeyQueryEmbedDim).
            SetKeyMask(mask).SetQueryMask(mask).
            SetOutputDim(embedDim).
            SetValueHeadDim(embedDim).Done()
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Normalize(ctx.In("normalization_1"), embed)
        attentionOutput := embed

        // Transformers recipe: 2 dense layers after attention.
        embed = layers.Dense(ctx.In("ffn_1"), embed, true, embedDim)
        embed = Tanh(embed)
        embed = layers.Dense(ctx.In("ffn_2"), embed, true, embedDim)
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Add(embed, attentionOutput)
        embed = Normalize(ctx.In("normalization_2"), embed)

        // Residual connection: not part of the usual transfomer layer ...
        if ii > 0 {
            embed = Add(residual, embed)
        }
    }
    return embed
}

%% --model=transformer --steps=3000 --batch=100 --optimizer=adamw --max_len=200 --max_att_len=200 --att_heads=4 --att_layers=3 --att_key_dim=32 --max_vocab=10000
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (3000 steps):  100% [========================================] (7 steps/s) [loss=0.327] [~loss=0.166] [~acc=94.78%]          


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.308731e+01 L3.437549e+01,7.392128e+00 L5.135394e+01,7.392128e+00 L6.833240e+01,1.345268e+01 L8.531086e+01,5.200000e+00 L1.022893e+02,6.274567e+00 L1.192678e+02,5.985239e+01 L1.362462e+02,9.640910e+01 L1.532247e+02,1.611194e+02 L1.702032e+02,1.636339e+02 L1.871816e+02,1.771949e+02 L2.041601e+02,1.816866e+02 L2.211385e+02,1.822024e+02 L2.381170e+02,1.829546e+02 L2.550955e+02,1.870379e+02 L2.720739e+02,1.869519e+02 L2.890524e+02,1.820090e+02 L3.060308e+02,1.912502e+02 L3.230093e+02,1.919809e+02 L3.399878e+02,1.916371e+02 L3.569662e+02,1.909064e+02 L3.739447e+02,1.932059e+02 L3.909231e+02,1.943235e+02 L4.079016e+02,1.938077e+02 L4.248801e+02,1.973967e+02 L4.418585e+02,1.936358e+02 L4.588370e+02,1.963222e+02 L4.758154e+02,1.955270e+02 L4.927939e+02,1.980844e+02 L5.097724e+02,1.976761e+02 L5.267508e+02,1.988152e+02 L5.437293e+02,1.963007e+02 L5.607077e+02,1.970099e+02 L5.776862e+02,1.989871e+02 L5.946646e+02,1.984283e+02 L6.116431e+02,1.956774e+02 L6.286216e+02,1.981274e+02 L6.456000e+02,1.976976e+02 L6.625785e+02,1.981274e+02 L6.795569e+02,1.965371e+02 L6.965354e+02,1.978910e+02 L7.135139e+02,1.968809e+02 L7.304923e+02,1.964081e+02 L7.474708e+02,1.963651e+02 L7.644492e+02,1.950542e+02 L7.814277e+02,1.984283e+02 L7.984062e+02,1.968165e+02 L8.153846e+02,1.898748e+02 L8.323631e+02,1.979340e+02 L8.493415e+02,1.965801e+02 L8.663200e+02,1.944094e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.123908e+01 L3.437549e+01,7.392128e+00 L5.135394e+01,7.392128e+00 L6.833240e+01,1.431231e+01 L8.531086e+01,3.406284e+01 L1.022893e+02,3.492247e+01 L1.192678e+02,1.068754e+02 L1.362462e+02,1.238965e+02 L1.532247e+02,1.831265e+02 L1.702032e+02,1.889506e+02 L1.871816e+02,1.987722e+02 L2.041601e+02,2.044029e+02 L2.211385e+02,2.090880e+02 L2.381170e+02,2.122902e+02 L2.550955e+02,2.157288e+02 L2.720739e+02,2.186516e+02 L2.890524e+02,2.145898e+02 L3.060308e+02,2.212521e+02 L3.230093e+02,2.251205e+02 L3.399878e+02,2.272481e+02 L3.569662e+02,2.270762e+02 L3.739447e+02,2.247981e+02 L3.909231e+02,2.278284e+02 L4.079016e+02,2.307512e+02 L4.248801e+02,2.343833e+02 L4.418585e+02,2.347486e+02 L4.588370e+02,2.357157e+02 L4.758154e+02,2.372631e+02 L4.927939e+02,2.400999e+02 L5.097724e+02,2.397131e+02 L5.267508e+02,2.417763e+02 L5.437293e+02,2.410456e+02 L5.607077e+02,2.408951e+02 L5.776862e+02,2.433237e+02 L5.946646e+02,2.442908e+02 L6.116431e+02,2.362530e+02 L6.286216e+02,2.468912e+02 L6.456000e+02,2.453438e+02 L6.625785e+02,2.474070e+02 L6.795569e+02,2.488899e+02 L6.965354e+02,2.496206e+02 L7.135139e+02,2.500289e+02 L7.304923e+02,2.509531e+02 L7.474708e+02,2.484601e+02 L7.644492e+02,2.509960e+02 L7.814277e+02,2.527798e+02 L7.984062e+02,2.520921e+02 L8.153846e+02,2.413249e+02 L8.323631e+02,2.532311e+02 L8.493415e+02,2.548000e+02 L8.663200e+02,2.512109e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.276495e+01 L3.437549e+01,3.183851e+01 L5.135394e+01,2.211099e+01 L6.833240e+01,2.372198e+01 L8.531086e+01,2.222689e+01 L1.022893e+02,2.257112e+01 L1.192678e+02,3.182941e+01 L1.362462e+02,6.992167e+01 L1.532247e+02,1.096124e+02 L1.702032e+02,1.377364e+02 L1.871816e+02,1.588245e+02 L2.041601e+02,1.748726e+02 L2.211385e+02,1.855009e+02 L2.381170e+02,1.923912e+02 L2.550955e+02,1.985957e+02 L2.720739e+02,2.043888e+02 L2.890524e+02,2.078204e+02 L3.060308e+02,2.096311e+02 L3.230093e+02,2.158932e+02 L3.399878e+02,2.186026e+02 L3.569662e+02,2.193720e+02 L3.739447e+02,2.206484e+02 L3.909231e+02,2.247332e+02 L4.079016e+02,2.245812e+02 L4.248801e+02,2.262981e+02 L4.418585e+02,2.281385e+02 L4.588370e+02,2.306016e+02 L4.758154e+02,2.321302e+02 L4.927939e+02,2.330102e+02 L5.097724e+02,2.319493e+02 L5.267508e+02,2.337143e+02 L5.437293e+02,2.364630e+02 L5.607077e+02,2.363771e+02 L5.776862e+02,2.353635e+02 L5.946646e+02,2.361096e+02 L6.116431e+02,2.383462e+02 L6.286216e+02,2.389335e+02 L6.456000e+02,2.408816e+02 L6.625785e+02,2.413149e+02 L6.795569e+02,2.418067e+02 L6.965354e+02,2.421099e+02


Results on train-eval:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.518296e+02 L3.437549e+01,2.520187e+02 L5.135394e+01,2.523458e+02 L6.833240e+01,2.518219e+02 L8.531086e+01,2.521485e+02 L1.022893e+02,2.522214e+02 L1.192678e+02,2.481480e+02 L1.362462e+02,2.231399e+02 L1.532247e+02,1.728121e+02 L1.702032e+02,1.633404e+02 L1.871816e+02,1.473213e+02 L2.041601e+02,1.407138e+02 L2.211385e+02,1.379356e+02 L2.381170e+02,1.364091e+02 L2.550955e+02,1.307615e+02 L2.720739e+02,1.308669e+02 L2.890524e+02,1.374108e+02 L3.060308e+02,1.244048e+02 L3.230093e+02,1.256852e+02 L3.399878e+02,1.255797e+02 L3.569662e+02,1.256030e+02 L3.739447e+02,1.230227e+02 L3.909231e+02,1.214190e+02 L4.079016e+02,1.170186e+02 L4.248801e+02,1.242368e+02 L4.418585e+02,1.242076e+02 L4.588370e+02,1.250894e+02 L4.758154e+02,1.190463e+02 L4.927939e+02,1.166800e+02 L5.097724e+02,1.161927e+02 L5.267508e+02,1.250306e+02 L5.437293e+02,1.254862e+02 L5.607077e+02,1.301114e+02 L5.776862e+02,1.219040e+02 L5.946646e+02,1.271220e+02 L6.116431e+02,1.325633e+02 L6.286216e+02,1.217024e+02 L6.456000e+02,1.231343e+02 L6.625785e+02,1.323628e+02 L6.795569e+02,1.225145e+02 L6.965354e+02,1.277690e+02 L7.135139e+02,1.288430e+02 L7.304923e+02,1.276525e+02 L7.474708e+02,1.381114e+02 L7.644492e+02,1.300047e+02 L7.814277e+02,1.257756e+02 L7.984062e+02,1.313672e+02 L8.153846e+02,1.566850e+02 L8.323631e+02,1.477614e+02 L8.493415e+02,1.422373e+02 L8.663200e+02,1.486071e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.519584e+02 L3.437549e+01,2.519677e+02 L5.135394e+01,2.519120e+02 L6.833240e+01,2.512733e+02 L8.531086e+01,2.504751e+02 L1.022893e+02,2.498179e+02 L1.192678e+02,2.424358e+02 L1.362462e+02,2.017861e+02 L1.532247e+02,1.502956e+02 L1.702032e+02,1.384985e+02 L1.871816e+02,1.251966e+02 L2.041601e+02,1.170675e+02 L2.211385e+02,1.094173e+02 L2.381170e+02,1.035031e+02 L2.550955e+02,9.873648e+01 L2.720739e+02,9.333899e+01 L2.890524e+02,9.626639e+01 L3.060308e+02,8.730844e+01 L3.230093e+02,8.231245e+01 L3.399878e+02,7.886278e+01 L3.569662e+02,7.751521e+01 L3.739447e+02,8.139599e+01 L3.909231e+02,7.409092e+01 L4.079016e+02,7.186149e+01 L4.248801e+02,6.643312e+01 L4.418585e+02,6.566948e+01 L4.588370e+02,6.416074e+01 L4.758154e+02,6.441688e+01 L4.927939e+02,6.049821e+01 L5.097724e+02,5.979095e+01 L5.267508e+02,5.511873e+01 L5.437293e+02,5.470033e+01 L5.607077e+02,5.409491e+01 L5.776862e+02,5.214476e+01 L5.946646e+02,5.139589e+01 L6.116431e+02,6.293744e+01 L6.286216e+02,4.834216e+01 L6.456000e+02,5.012286e+01 L6.625785e+02,4.534142e+01 L6.795569e+02,4.554270e+01 L6.965354e+02,4.360538e+01 L7.135139e+02,4.231143e+01 L7.304923e+02,4.102879e+01 L7.474708e+02,4.266514e+01 L7.644492e+02,4.138905e+01 L7.814277e+02,4.113882e+01 L7.984062e+02,3.928023e+01 L8.153846e+02,4.991826e+01 L8.323631e+02,3.807195e+01 L8.493415e+02,3.556967e+01 L8.663200e+02,3.802682e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L3.437549e+01,2.493059e+02 L5.135394e+01,2.483765e+02 L6.833240e+01,2.480077e+02 L8.531086e+01,2.532145e+02 L1.022893e+02,2.523015e+02 L1.192678e+02,2.419460e+02 L1.362462e+02,2.119904e+02 L1.532247e+02,1.430053e+02 L1.702032e+02,1.450059e+02 L1.871816e+02,1.366942e+02 L2.041601e+02,1.090064e+02 L2.211385e+02,1.321441e+02 L2.381170e+02,1.219338e+02 L2.550955e+02,1.133116e+02 L2.720739e+02,9.375723e+01 L2.890524e+02,1.076350e+02 L3.060308e+02,1.022890e+02 L3.230093e+02,9.172398e+01 L3.399878e+02,1.217370e+02 L3.569662e+02,8.690090e+01 L3.739447e+02,5.197982e+01 L3.909231e+02,4.943265e+01 L4.079016e+02,1.031499e+02 L4.248801e+02,6.951271e+01 L4.418585e+02,3.420414e+01 L4.588370e+02,9.465873e+01 L4.758154e+02,7.575688e+01 L4.927939e+02,7.044972e+01 L5.097724e+02,4.188760e+01 L5.267508e+02,5.045926e+01 L5.437293e+02,6.636493e+01 L5.607077e+02,3.899050e+01 L5.776862e+02,9.352333e+01 L5.946646e+02,6.965100e+01 L6.116431e+02,2.717566e+01 L6.286216e+02,6.602989e+01 L6.456000e+02,5.133402e+01 L6.625785e+02,5.396882e+01 L6.795569e+02,4.646746e+01 L6.965354e+02,3.326324e+01

	Mean Loss (#loss): 0.147
	Mean Accuracy (#acc): 95.38%
Results on test-eval:
	Mean Loss (#loss): 0.429
	Mean Accuracy (#acc): 84.81%
